In [1]:
import numpy as np 
import pandas as pd

from sklearn.preprocessing import MinMaxScaler, OneHotEncoder    # 特徵縮放、編碼，為計算其變異數
from sklearn.compose import make_column_transformer    # 建立流程
from sklearn.feature_selection import VarianceThreshold    # 移除低變異特徵

### Data Import

In [3]:
dict_df_pre = pd.read_pickle('data_pre.pkl')
dict_df_pre.keys()

dict_keys(['target', 'features', 'others', 'features_pre'])

In [4]:
X = dict_df_pre['features_pre'].copy()

### Remove Low-variance Features by Variance Threshold